<a href="https://colab.research.google.com/github/maheshwari1394/demo/blob/main/cnn_finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow scikit-learn pandas

import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.utils import class_weight
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam



In [ ]:
# Load your dataset
csv_path = "/content/FinalDatasetwithprobabilities_utf8sig (2).csv"
df = pd.read_csv(csv_path)

print("🔎 Dataset shape:", df.shape)
print(df.head())


🔎 Dataset shape: (27355, 3)
                                        Conversation          Label  \
0  Beta, this is Papa calling from my friend's ph...  This is fraud   
1  Son, I am in police custody in Pune. They caug...  This is fraud   
2  This is your sister Kavya speaking. My phone g...  This is fraud   
3  Beta Sneha, this is your Nana calling from hos...  This is fraud   
4  Son, this is Papa. I am trapped in a business ...  This is fraud   

   scam_probability  
0             89.94  
1             86.80  
2             81.43  
3             87.33  
4             95.26  


In [ ]:
# Use numeric features only
X = df[['scam_probability']].values   # Feature column

# Map string labels -> numeric
label_mapping = {
    "This is fraud": 1,
    "Not fraud": 0,
    "Fraud": 1,
    "Normal": 0
}

# Apply mapping
y = df['Label'].map(label_mapping)

# Drop rows with missing labels
mask = y.notna()
X = X[mask]
y = y[mask].astype(int)


In [ ]:
print("Unique labels after cleaning:", np.unique(y))
print("Dataset size:", len(y))


Unique labels after cleaning: [1]
Dataset size: 2265


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Scale features
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)


In [ ]:
from sklearn.utils import class_weight

print("🔎 Calculating class weights...")
class_weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=np.unique(y_train),
    y=y_train
)
class_weights = dict(enumerate(class_weights))
print("📊 Class Weights:", class_weights)


🔎 Calculating class weights...
📊 Class Weights: {0: np.float64(1.0)}


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(1, activation='sigmoid')
])

model.compile(
    optimizer=Adam(learning_rate=1e-4),
    loss='binary_crossentropy',
    metrics=['accuracy', tf.keras.metrics.AUC(), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [ ]:
# Make sure everything is numeric
X_train = np.array(X_train).astype("float32")
X_val = np.array(X_val).astype("float32")
y_train = np.array(y_train).astype(int)
y_val = np.array(y_val).astype(int)

print("X_train shape:", X_train.shape, "dtype:", X_train.dtype)
print("y_train unique values:", np.unique(y_train))

callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint("best_model.keras", save_best_only=True)
]

history = model.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=20,
    batch_size=32,
    class_weight=class_weights,
    callbacks=callbacks,
    verbose=1
)


X_train shape: (1812, 1) dtype: float32
y_train unique values: [1]
Epoch 1/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 15s 139ms/step - accuracy: 0.5767 - auc: 0.0000e+00 - loss: 0.6896 - precision: 1.0000 - recall: 0.5767 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.6354 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 2/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9236 - auc: 0.0000e+00 - loss: 0.6294 - precision: 1.0000 - recall: 0.9236 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.5742 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 3/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9956 - auc: 0.0000e+00 - loss: 0.5731 - precision: 1.0000 - recall: 0.9956 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - val_loss: 0.5182 - val_precision: 1.0000 - val_recall: 1.0000
Epoch 4/20
57/57 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.9987 - auc: 0.0000e+00 - loss: 0.5193 - precision: 1.0000 - recall: 0.9987 - val_accuracy: 1.0000 - val_auc: 0.0000e+00 - 

In [ ]:
model.save("fraud_detector_final.keras")
print("✅ Final model saved: fraud_detector_final.keras")


✅ Final model saved: fraud_detector_final.keras


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Save model directly to Drive
model.save("/content/drive/MyDrive/fraud_detector_final.keras")
print("✅ Model saved in Google Drive (Keras format)")


Mounted at /content/drive
✅ Model saved in Google Drive (Keras format)


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

# Load saved model
model = keras.models.load_model("fraud_detector_final.keras")

# Check model input shape
print("Model input shape:", model.input_shape)

# ---------- CASE 1: Model trained with 1 feature (e.g., scam_probability only) ----------
if model.input_shape[-1] == 1:
    # Example input (replace with real feature)
    sample_input = np.array([[0.56]], dtype=np.float32)  # shape (1,1)

# ---------- CASE 2: Model trained with multiple features (e.g., 10 features) ----------
elif model.input_shape[-1] == 10:
    # Example input with 10 features
    sample_input = np.array([[0.12, 0.33, 0.56, 0.78, 0.44,
                              0.67, 0.23, 0.89, 0.45, 0.11]], dtype=np.float32)  # shape (1,10)

else:
    raise ValueError(f"Unexpected input shape: {model.input_shape}")

# Run prediction
prediction = model.predict(sample_input)

# Since it's binary classification, take probability
fraud_probability = prediction[0][0]

label = "Fraud" if fraud_probability >= 0.5 else "Normal"

print("Raw prediction:", prediction)
print("Fraud probability:", fraud_probability)
print("Predicted label:", label)


Model input shape: (None, 1)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 218ms/step
Raw prediction: [[0.9391629]]
Fraud probability: 0.9391629
Predicted label: Fraud


In [ ]:
!git config --global user.name "maheshwari1394"
!git config --global user.email "maheshwaripatil1394@gmail.com"


In [ ]:
!git clone https://github.com/maheshwari1394/cnn_finetune.git


Cloning into 'cnn_finetune'...


In [ ]:
!pwd



/content


In [ ]:
!ls


cnn_finetune  sample_data


In [ ]:
!git status



fatal: not a git repository (or any of the parent directories): .git


In [ ]:
%cd /content/cnn_finetune


/content/cnn_finetune


In [ ]:
!ls -a


.  ..  .git


In [ ]:
!git status


On branch main

No commits yet

nothing to commit (create/copy files and use "git add" to track)


In [ ]:
!git commit -m "Add CNN finetuning code"


On branch main

Initial commit

nothing to commit (create/copy files and use "git add" to track)


In [ ]:
!find /content -name "*.ipynb"


